In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_May7_ntuple_old/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)
    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr,br=['px','py','pz','e'],name="LV"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)
    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def mass(self):
     #   r"""Invariant mass (+, -, -, -)
      #  :math:`\sqrt{t^2-x^2-y^2-z^2}`
      #  """
        return numpy.sqrt(self.mass2)
    
def absolute(self):
      #  """Magnitude of this Lorentz vector
     #   Alias for `mass`
       # """
        return self.mass
    
def flat(x,axis=None):
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
hists = {}
hists["Target_mass_hist"] = hist.Hist("Target SP Kaons",
                                hist.Cat("Ptype", "Type of Particle"),         
                                hist.Bin("m", r"Mass", 40, 0, 3000),
                            )                                      

In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    target_kaon_dau1 = ldmx_events['Target_Kaon_dau1']
    target_kaon_dau2 = ldmx_events['Target_Kaon_dau2']
    
    masks_id = {'Pionp': 211,
                'Pionm': -211,
               }

    for part,pmask in masks_id.items():
        part_mask1p = ((target_kaon_dau1.pdgID)==pmask)
        part_mask2p = ((target_kaon_dau2.pdgID)==pmask)
        part_mask1k = (abs(target_kaon_dau1.mompdgID)== 310)
        part_mask2k = (abs(target_kaon_dau2.mompdgID)== 310)
        
        target_kaon_dau1a = get_vector(target_kaon_dau1[part_mask1p & part_mask1k][:15])
        target_kaon_dau2a = get_vector(target_kaon_dau2[part_mask2p & part_mask2k][:15])
    
        print(target_kaon_dau1a.mass)
        print(target_kaon_dau2a.mass)
        
        sum_kaon = target_kaon_dau1a + target_kaon_dau2a
        print(sum_kaon)
        #sum_kaon.sum(target_kaon_dau2a)
    
        hists["Target_mass_hist"].fill(Ptype=part,      
                                 m=sum_kaon.mass,
                                )

In [6]:
nchunk = 0
for chunk in ldmx_dict:
    #I usually test my code in one chunck only
    if nchunk >= 1: break
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

process 1
[[], [], [140], [], [], [], [], [], [], [], [], [], [], [140], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


ValueError: in ListOffsetArray64, cannot broadcast nested list

(https://github.com/scikit-hep/awkward-1.0/blob/1.2.3/src/cpu-kernels/awkward_ListArray_broadcast_tooffsets.cpp#L27)

In [ ]:
print(hists["Target_mass_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["Target_mass_hist"],ax=ax,clear=False);

